In [1]:
import os
import sys
import numpy as np
import pandas as pd

In [2]:
import nltk
import spacy

In [3]:
! pip install wikipedia-api

  Created wheel for wikipedia-api: filename=Wikipedia_API-0.5.4-py3-none-any.whl size=13462 sha256=6dc8cbb8548b8ec66255a93787c008df395f2c90e64f9e8f561038fa8c4f1fbb
  Stored in directory: /Users/rajabiswas/Library/Caches/pip/wheels/d3/24/56/58ba93cf78be162451144e7a9889603f437976ef1ae7013d04
Successfully built wikipedia-api


In [4]:
import wikipediaapi  # pip install wikipedia-api
import pandas as pd
import concurrent.futures
from tqdm import tqdm

def wiki_scrape(topic_name, verbose=True):
    def wiki_link(link):
        try:
            page = wiki_api.page(link)
            if page.exists():
                return {'page': link, 'text': page.text, 'link': page.fullurl,
                        'categories': list(page.categories.keys())}
        except:
            return None

    wiki_api = wikipediaapi.Wikipedia(language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI)
    page_name = wiki_api.page(topic_name)
    if not page_name.exists():
        print('Page {} does not exist.'.format(topic_name))
        return
    
    page_links = list(page_name.links.keys())
    progress = tqdm(desc='Links Scraped', unit='', total=len(page_links)) if verbose else None
    sources = [{'page': topic_name, 'text': page_name.text, 'link': page_name.fullurl,
                'categories': list(page_name.categories.keys())}]
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        future_link = {executor.submit(wiki_link, link): link for link in page_links}
        for future in concurrent.futures.as_completed(future_link):
            data = future.result()
            sources.append(data) if data else None
            progress.update(1) if verbose else None     
    progress.close() if verbose else None
    
    namespaces = ('Wikipedia', 'Special', 'Talk', 'LyricWiki', 'File', 'MediaWiki',
                  'Template', 'Help', 'User', 'Category talk', 'Portal talk')
    sources = pd.DataFrame(sources)
    sources = sources[(len(sources['text']) > 20)
                      & ~(sources['page'].str.startswith(namespaces, na=True))]
    sources['categories'] = sources.categories.apply(lambda x: [y[9:] for y in x])
    sources['topic'] = topic_name
    print('Wikipedia pages scraped:', len(sources))
    
    return sources

In [5]:
wiki_data = wiki_scrape('self-driving technology')

Links Scraped: 100%|██████████| 917/917 [03:35<00:00,  4.25/s]

Wikipedia pages scraped: 889


In [6]:
wiki_data.head()

,page,text,link,categories,topic
0,self-driving technology,"A self-driving car, also known as an autonomou...",https://en.wikipedia.org/wiki/Self-driving_car,[All articles containing potentially dated sta...,self-driving technology
1,17-4 stainless steel,SAE Type 630 stainless steel (more commonly kn...,https://en.wikipedia.org/wiki/17-4_stainless_s...,"[All stub articles, Alloy stubs, CS1 maint: di...",self-driving technology
2,163.com,%5B%5BWikipedia%3ARedirects+for+discussion%5D%...,https://en.wikipedia.org/wiki/163.com,"[All redirects for discussion, Printworthy red...",self-driving technology
3,2+2 (car body style),The 2+2 is a version of the coupé car-body sty...,https://en.wikipedia.org/wiki/2%2B2_(car_body_...,"[All articles needing additional references, A...",self-driving technology
4,2022 Winter Olympics,"The 2022 Winter Olympics, officially known as ...",https://en.wikipedia.org/wiki/2022_Winter_Olym...,"[2020s in Beijing, 2022 Winter Olympics, 2022 ...",self-driving technology


In [9]:
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
import itertools

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rajabiswas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
text = wiki_data.loc[0,'text']
text = text.split('\n')
text = [sent for sent in text if len(sent)>=2]
text = [sent_tokenize(sent) for sent in text]
text = list(itertools.chain(*text))
text = [sent for sent in text if len(sent)>=3]
sentences = [sent.strip().lower() for sent in text]
sentences[0]

'a self-driving car, also known as an autonomous vehicle (av or auto), driverless car, or robo-car, is a vehicle that is capable of sensing its environment and moving safely with little or no human input.self-driving cars combine a variety of sensors to perceive their surroundings, such as radar, lidar, sonar, gps, odometry and inertial measurement units.'

In [18]:
nlp = spacy.load('en_core_web_sm')
# neuralcoref.add_to_pipe(nlp)

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
sent = sentences[0]
sent = nlp(sent)
spans = list(sent.ents) + list(sent.noun_chunks)  # collect nodes
spans = spacy.util.filter_spans(spans)

In [19]:
spacy.__version__

'3.0.3'